<a href="https://colab.research.google.com/github/Bindusri29/project2/blob/main/AdaBoost2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
import random
import copy
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import warnings
warnings.filterwarnings('ignore')

phases = ['training', 'validation']

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


# TL model for feature extraction
class ConvNet(nn.Module):
    def __init__(self, model, num_classes):
        super(ConvNet, self).__init__()
        self.base_model = nn.Sequential(*list(model.children())[:-1])
        self.linear1 = nn.Linear(in_features=2048, out_features=512)
        self.linear2 = nn.Linear(in_features=512, out_features=num_classes)
        self.relu = nn.LeakyReLU()

  #For Grad-CAM
    def activations_hook(self, grad):
        self.gradients = grad

    def get_activations_gradient(self):
        return self.gradients

    # method for the activation exctraction
    def get_activations(self, x):
        return self.base_model[:-1](x)

    def forward(self, x):
        x = self.base_model(x)
        #h = x.register_hook(self.activations_hook) #Grad-CAM
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        lin = self.relu(x)
        x = self.linear2(lin)
        return lin, x


# utility function for training CNN model
def train_model(model, criterion, optimizer, scheduler, data_loader, batch_size, num_epochs=30):

    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0

    for epoch in range(num_epochs):

        for phase in phases:
            if phase == 'training':
                model.train()
            else:
                model.eval()

            epoch_loss = 0
            epoch_corrects = 0

            for ii, (images, labels) in enumerate(data_loader[phase]):

                images = images.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase == 'training'):
                    _, outputs = model(images)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'training':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                epoch_corrects += torch.sum(preds == labels.data)
                epoch_loss += loss.item() * images.size(0)

            epoch_accuracy = epoch_corrects/len(data_loader[phase])
            epoch_loss /= len(data_loader[phase])

            # store statistics
            if phase == 'training':
                train_loss.append(epoch_loss)
                epoch_accuracy = epoch_accuracy / float(batch_size)
                train_acc.append(epoch_accuracy)
                scheduler.step()
            if phase == 'validation':
                val_loss.append(epoch_loss)
                val_acc.append(epoch_accuracy)

            print(f'Epoch: [{epoch+1}/{num_epochs}] Phase: {phase} | Loss: {epoch_loss:.6f} Accuracy: {epoch_accuracy:.6f}')

            # deep copy the best model weights
            if phase == 'validation' and epoch_accuracy >= best_accuracy:
                best_accuracy = epoch_accuracy
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'====> Best accuracy reached so far at Epoch {epoch+1} Accuracy = {best_accuracy:.6f}')

        print('-------------------------------------------------------------------------')

    # training complete
    print(f'Best Validation Accuracy: {best_accuracy:4f}')
    model.load_state_dict(best_model_wts)

    history = {
        'train_loss': train_loss.copy(),
        'train_acc': train_acc.copy(),
        'val_loss': val_loss.copy(),
        'val_acc': val_acc.copy()
    }

    return model, history


# utility function to evaluate model on dataset and extract features
def eval_model_extract_features(features, true_labels, model, dataloader, phase):

    conf_mat = torch.zeros(2, 2)

    with torch.no_grad():
        # for entire dataset
        n_correct = 0
        n_samples = 0

        model.eval()

        for images, labels in dataloader:

            images = images.to(device)
            labels = labels.to(device)

            true_labels.append(labels)

            ftrs, outputs = model(images)
            features.append(ftrs)

            _, preds = torch.max(outputs, 1)

            for t, p in zip(labels.view(-1), preds.view(-1)):
                conf_mat[t.long()][p.long()] += 1

            n_samples += labels.size(0)
            n_correct += (preds == labels).sum().item()

        accuracy = n_correct/float(n_samples)

        print(f'Accuracy of model on {phase} set = {(100.0 * accuracy):.4f} %')

    print(conf_mat)
    print(conf_mat.diag()/conf_mat.sum(1))
    return features, true_labels


def get_features(features, true_labels):
    ftrs = features.copy()
    lbls = true_labels.copy()

    for i in range(len(ftrs)):
        ftrs[i] = ftrs[i].cpu().numpy()

    for i in range(len(lbls)):
        lbls[i] = lbls[i].cpu().numpy()

    # convert to numpy array
    ftrs = np.array(ftrs)
    lbls = np.array(lbls)

    n_samples = ftrs.shape[0] * ftrs.shape[1]
    n_features = ftrs.shape[2]
    ftrs = ftrs.reshape(n_samples, n_features)

    n_lbls = lbls.shape[0]
    lbls = lbls.reshape(n_lbls)

    return ftrs, lbls

In [ ]:
pip install adaboost

ERROR: Could not find a version that satisfies the requirement adaboost (from versions: none)
ERROR: No matching distribution found for adaboost


In [ ]:

import os
import sys
import random
import math
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier as KNN
#from sklearn import SVM
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC  # for classification
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
import numpy as np

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


# model class for a FS solution
class Solution():
    # structure of the solution
    def __init__(self):
        self.num_features = None
        self.num_agents = None
        self.max_iter = None
        self.obj_function = None
        self.execution_time = None
        self.convergence_curve = {}
        self.best_agent = None
        self.best_fitness = None
        self.best_accuracy = None
        self.final_population = None
        self.final_fitness = None
        self.final_accuracy = None


class Data():
    # structure of the training data
    def __init__(self):
        self.train_X = None
        self.train_Y = None
        self.val_X = None
        self.val_Y = None


def initialize(num_agents, num_features):
    # define min and max number of features
    min_features = int(0.3 * num_features)
    max_features = int(0.5 * num_features)

    # initialize the agents with zeros
    agents = np.zeros((num_agents, num_features))

    # select random features for each agent
    for agent_no in range(num_agents):

        random.seed(time.time() + agent_no)

        num = random.randint(min_features, max_features)
        pos = random.sample(range(0, num_features - 1), num)

        for idx in pos:
            agents[agent_no][idx] = 1

    return agents


def sort_agents(agents, obj, data):
    # sort the agents according to fitness
    train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj

    # if there is only one agent
    if len(agents.shape) == 1:
        num_agents = 1
        fitness, acc = obj_function(
            agents, train_X, val_X, train_Y, val_Y, weight_acc)
        return agents, fitness, acc

    # for multiple agents
    else:
        num_agents = agents.shape[0]
        fitness = np.zeros(num_agents)
        acc = np.zeros(num_agents)
        for id, agent in enumerate(agents):
            fitness[id], acc[id] = obj_function(
                agent, train_X, val_X, train_Y, val_Y, weight_acc)
        idx = np.argsort(-fitness)
        sorted_agents = agents[idx].copy()
        sorted_fitness = fitness[idx].copy()
        sorted_acc = acc[idx].copy()

    return sorted_agents, sorted_fitness, sorted_acc


def display(agents, fitness, acc, agent_name='Agent'):
    # display the population
    print('\nNumber of agents: {}'.format(agents.shape[0]))
    print('\n------------- Best Agent ---------------')
    print('Fitness: {}'.format(fitness[0]))
    print('Accuracy: {}'.format(acc[0]))
    print('Number of Features: {}'.format(int(np.sum(agents[0]))))
    print('----------------------------------------\n')

    for id, agent in enumerate(agents):
        print('{} {} - Fitness: {},Accuracy: {}, Number of Features: {}'.format(agent_name, id+1, fitness[id], acc[id], int(np.sum(agent))))

    print('================================================================================\n')


def compute_accuracy(agent, train_X, test_X, train_Y, test_Y):
    # compute classification accuracy of the given agents
    cols = np.flatnonzero(agent)
    if(cols.shape[0] == 0):
        return 0

    #clf = KNN()
    #clf=svm.SVC(kernel='linear')
    #clf = XGBClassifier()
    clf = AdaBoostClassifier()  # Instantiate AdaBoostClassifier
    train_data = train_X[:, cols]
    train_label = train_Y
    test_data = test_X[:, cols]
    test_label = test_Y

    clf.fit(train_data, train_label)
    acc = clf.score(test_data, test_label)

    return acc


def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.9, dims=None):
    # compute a basic fitness measure
    if(weight_acc == None):
        weight_acc = 0.99
    weight_feat = 1 - weight_acc

    if dims != None:
        num_features = dims
    else:
        num_features = agent.shape[0]

    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent))/num_features

    fitness = weight_acc * acc + weight_feat * feat

    return fitness, acc


def sigmoid(val):
    if val < 0:
        return 1 - 1/(1 + np.exp(val))
    else:
        return 1/(1 + np.exp(-val))


def get_trans_function(shape='s'):
    return sigmoid


def relu(X):
    return np.maximum(0, X)



def validate_FS(X, y, agent):
    cols = np.flatnonzero(agent)
    if cols.shape[0] == 0:
        return 0

    X1 = X[:, cols].copy()  # getting selected features

    X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, shuffle=True)

    # Use KNeighborsClassifier instead of svm.SVC for k-NN
    #model =svm.SVC(kernel='linear',probability=True)
    #model = XGBClassifier()
    model = AdaBoostClassifier()
    #model=KNN()


    model.fit(X_train, y_train)

    # Assuming y_pred_binary is your binary prediction
    y_pred_binary = model.predict(X_test)
    accuracy = np.sum(y_pred_binary == y_test) / y_test.shape[0]

    # Code for ROC
    y_scores = model.predict_proba(X_test)[:, 1]  # probability estimates for the positive class
    fpr, tpr, _ = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()

    print('-' * 50)
    print(classification_report(y_test, y_pred_binary, digits=4))
    print('-' * 50)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred_binary)

    # Plot the confusion matrix using seaborn
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title('Confusion matrix of the classifier')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    print(f'True Positives = {tp} \nFalse Positives = {fp} \nFalse Negatives = {fn} \nTrue Negatives = {tn}')

In [ ]:
import os
import sys
import random
import math
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, classification_report, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier  # Updated import

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# model class for a FS solution
class Solution():
    def __init__(self):
        self.num_features = None
        self.num_agents = None
        self.max_iter = None
        self.obj_function = None
        self.execution_time = None
        self.convergence_curve = {}
        self.best_agent = None
        self.best_fitness = None
        self.best_accuracy = None
        self.final_population = None
        self.final_fitness = None
        self.final_accuracy = None

class Data():
    def __init__(self):
        self.train_X = None
        self.train_Y = None
        self.val_X = None
        self.val_Y = None

def initialize(num_agents, num_features):
    min_features = int(0.3 * num_features)
    max_features = int(0.5 * num_features)
    agents = np.zeros((num_agents, num_features))

    for agent_no in range(num_agents):
        random.seed(time.time() + agent_no)
        num = random.randint(min_features, max_features)
        pos = random.sample(range(0, num_features - 1), num)
        for idx in pos:
            agents[agent_no][idx] = 1

    return agents

def sort_agents(agents, obj, data):
    train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj

    if len(agents.shape) == 1:
        num_agents = 1
        fitness, acc = obj_function(agents, train_X, val_X, train_Y, val_Y, weight_acc)
        return agents, fitness, acc
    else:
        num_agents = agents.shape[0]
        fitness = np.zeros(num_agents)
        acc = np.zeros(num_agents)
        for id, agent in enumerate(agents):
            fitness[id], acc[id] = obj_function(agent, train_X, val_X, train_Y, val_Y, weight_acc)
        idx = np.argsort(-fitness)
        sorted_agents = agents[idx].copy()
        sorted_fitness = fitness[idx].copy()
        sorted_acc = acc[idx].copy()

    return sorted_agents, sorted_fitness, sorted_acc

def display(agents, fitness, acc, agent_name='Agent'):
    print('\nNumber of agents: {}'.format(agents.shape[0]))
    print('\n------------- Best Agent ---------------')
    print('Fitness: {}'.format(fitness[0]))
    print('Accuracy: {}'.format(acc[0]))
    print('Number of Features: {}'.format(int(np.sum(agents[0]))))
    print('----------------------------------------\n')

    for id, agent in enumerate(agents):
        print('{} {} - Fitness: {}, Accuracy: {}, Number of Features: {}'.format(agent_name, id+1, fitness[id], acc[id], int(np.sum(agent))))

    print('================================================================================\n')

def compute_accuracy(agent, train_X, test_X, train_Y, test_Y):
    cols = np.flatnonzero(agent)
    if(cols.shape[0] == 0):
        return 0

    clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')  # Updated classifier
    train_data = train_X[:, cols]
    train_label = train_Y
    test_data = test_X[:, cols]
    test_label = test_Y

    clf.fit(train_data, train_label)
    acc = clf.score(test_data, test_label)

    return acc

def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.9, dims=None):
    if weight_acc is None:
        weight_acc = 0.99
    weight_feat = 1 - weight_acc

    if dims is not None:
        num_features = dims
    else:
        num_features = agent.shape[0]

    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent)) / num_features

    fitness = weight_acc * acc + weight_feat * feat

    return fitness, acc

def sigmoid(val):
    if val < 0:
        return 1 - 1/(1 + np.exp(val))
    else:
        return 1/(1 + np.exp(-val))

def get_trans_function(shape='s'):
    return sigmoid

def relu(X):
    return np.maximum(0, X)

def validate_FS(X, y, agent):
    cols = np.flatnonzero(agent)
    if cols.shape[0] == 0:
        return 0

    X1 = X[:, cols].copy()
    X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, shuffle=True)

    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')  # Updated classifier
    model.fit(X_train, y_train)

    y_pred_binary = model.predict(X_test)
    accuracy = np.sum(y_pred_binary == y_test) / y_test.shape[0]

    y_scores = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()

    print('-' * 50)
    print(classification_report(y_test, y_pred_binary, digits=4))
    print('-' * 50)

    cm = confusion_matrix(y_test, y_pred_binary)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title('Confusion matrix of the classifier')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    tn, fp, fn, tp = cm.ravel()
    print(f'True Positives = {tp} \nFalse Positives = {fp} \nFalse Negatives = {fn} \nTrue Negatives = {tn}')


In [ ]:
import numpy as np
import time
from math import e
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn import datasets
#from utils.feature_selection import *
#added new code
import matplotlib.pyplot as plt


def alturism(good_arr,bad_arr,good_vel,bad_vel,trans_func_shape='s'):
    trans_function = get_trans_function(trans_func_shape)
    for i in range(len(good_vel)):
        if good_vel[i]>0 and good_vel[i]<1.5:
            if np.random.random()<np.random.uniform(0.5,0.8):
                bad_arr[i]=good_arr[i]
                bad_vel[i]=good_vel[i]
                good_vel[i]=np.random.random()
                trans_value = trans_function(good_vel[i])
                if (np.random.random() < trans_value):
                    good_arr[i] = 1
                else:
                    good_arr[i] = 0
        else:
            if np.random.random()<0.5:
                bad_arr[i]=good_arr[i]
                bad_vel[i]=good_vel[i]
                good_vel[i]=np.random.random()
                trans_value = trans_function(good_vel[i])
                if (np.random.random() < trans_value):
                    good_arr[i] = 1
                else:
                    good_arr[i] = 0
    return good_arr,bad_arr,good_vel,bad_vel

def AAPSO(num_agents, max_iter, train_data, train_label, obj_function=compute_fitness, trans_func_shape='s', save_conv_graph=False):

    # Adaptive and Altruistic Particle Swarm Optimizer

    ############################### Parameters ####################################
    #                                                                             #
    #   num_agents: number of particles                                           #
    #   max_iter: maximum number of generations                                   #
    #   train_data: training samples of data                                      #
    #   train_label: class labels for the training samples                        #
    #   obj_function: the function to maximize while doing feature selection      #
    #   trans_function_shape: shape of the transfer function used                 #
    #   save_conv_graph: boolean value for saving convergence graph               #
    #                                                                             #
    ###############################################################################

    short_name = 'AAPSO'
    agent_name = 'Particle'
    train_data, train_label = np.array(train_data), np.array(train_label)
    num_features = train_data.shape[1]
    trans_function = get_trans_function(trans_func_shape)

    # setting up the objectives
    weight_acc = None
    if(obj_function==compute_fitness):

        obj = (obj_function, 0.98)
        compute_accuracy = (compute_fitness, 1) # compute_accuracy is just compute_fitness with accuracy weight as 1

        # initialize particles and Leader (the agent with the max fitness)
        particles = initialize(num_agents, num_features)
        fitness = np.zeros(num_agents)
        prev_fitness = np.zeros(num_agents)
        accuracy = np.zeros(num_agents)
        Leader_agent = np.zeros((1, num_features))
        Leader_fitness = float("-inf")
        Leader_accuracy = float("-inf")

        # initialize convergence curves
        convergence_curve = {}
        convergence_curve['fitness'] = np.zeros(max_iter)

        # initialize data class
        data = Data()
        data.train_X, data.val_X, data.train_Y, data.val_Y = train_test_split(train_data, train_label, stratify=train_label,shuffle=True, test_size=0.2)

        # create a solution object
        solution = Solution()
        solution.num_agents = num_agents
        solution.max_iter = max_iter
        solution.num_features = num_features
        solution.obj_function = obj_function

        # rank initial particles
        particles, fitness,acc = sort_agents(particles, obj, data)#added acc

        # start timer
        start_time = time.time()

        # initialize global and local best particles
        globalBestParticle = [0 for i in range(num_features)]
        globalBestFitness = float("-inf")
        localBestParticle = [ [ 0 for i in range(num_features) ] for j in range(num_agents) ]
        localBestFitness = [float("-inf") for i in range(num_agents) ]
        weight = 1.0
        velocity = [ [ 0 for i in range(num_features) ] for j in range(num_agents) ]

        for iter_no in range(max_iter):
            print('\n================================================================================')
            print('                          Iteration - {}'.format(iter_no+1))
            print('================================================================================\n')
            # update adaptive weight
            weight= 1-(e**-(1-iter_no/max_iter))
            prev_fitness=fitness
            # update the velocity
            for i in range(num_agents):
                for j in range(num_features):
                    velocity[i][j] = (weight*velocity[i][j])
                    r1, r2 = np.random.random(2)
                    velocity[i][j] = velocity[i][j] + (r1 * (localBestParticle[i][j] - particles[i][j]))
                    velocity[i][j] = velocity[i][j] + (r2 * (globalBestParticle[j] - particles[i][j]))

            # updating position of particles
            for i in range(num_agents):
                for j in range(num_features):
                    trans_value = trans_function(velocity[i][j])
                    if (np.random.random() < trans_value):
                        particles[i][j] = 1
                    else:
                        particles[i][j] = 0

            #alturism
            for i in range(num_agents):
                fitness[i],_=compute_fitness(particles[i], data.train_X, data.val_X, data.train_Y, data.val_Y, weight_acc)
            delta_fit=np.subtract(fitness,prev_fitness)
            alturism_rank=np.argsort(delta_fit)

            for i in range(int(0.3*num_agents)):
                good_idx=int((np.where(alturism_rank==(int(0.4*num_agents)+i+1)))[0])
                bad_idx=int((np.where(alturism_rank==num_agents-(i+1)))[0])
                particles[good_idx],particles[bad_idx],velocity[good_idx],velocity[bad_idx]=alturism(particles[good_idx],particles[bad_idx],velocity[good_idx],velocity[bad_idx])

            # updating fitness of particles
            particles, fitness, acc = sort_agents(particles, obj, data)#added acc
            display(particles, fitness,acc, agent_name)#changed the code


            # updating the global best and local best particles
            for i in range(num_agents):
                if fitness[i]>localBestFitness[i]:
                    localBestFitness[i]=fitness[i]
                    localBestParticle[i]=particles[i][:]

                if fitness[i]>globalBestFitness:
                    globalBestFitness=fitness[i]
                    globalBestParticle=particles[i][:]

            # update Leader (best agent)
            if globalBestFitness > Leader_fitness:
                Leader_agent = globalBestParticle.copy()
                Leader_fitness = globalBestFitness.copy()

            convergence_curve['fitness'][iter_no] = Leader_fitness

        # compute final accuracy
        Leader_agent, Leader_accuracy,_ = sort_agents(Leader_agent, compute_accuracy, data)
        particles, accuracy,_ = sort_agents(particles, compute_accuracy, data)

        print('\n================================================================================')
        print('                                    Final Result                                  ')
        print('================================================================================\n')
        print('Leader ' + agent_name + ' Dimension : {}'.format(int(np.sum(Leader_agent))))
        print('Leader ' + agent_name + ' Fitness : {}'.format(Leader_fitness))
        print('Leader ' + agent_name + ' Classification Accuracy : {}'.format(Leader_accuracy))
        print('\n================================================================================\n')

        # stop timer
        end_time = time.time()
        exec_time = end_time - start_time

        # plot convergence graph
        def plot_convergence_curve(convergence_curve):
            plt.plot(convergence_curve['fitness'])
            plt.xlabel('Iterations')
            plt.ylabel('Fitness')
            plt.title('Convergence Curve')
            plt.show()
        #fig, axes = Conv_plot(convergence_curve)
        #if(save_conv_graph):
            #plt.savefig('convergence_graph_'+ short_name + '.jpg')
        #plt.show()

        # update attributes of solution
        solution.best_agent = Leader_agent
        solution.best_fitness = Leader_fitness
        solution.best_accuracy = Leader_accuracy
        solution.convergence_curve = convergence_curve
        solution.final_particles = particles
        solution.final_fitness = fitness
        solution.final_accuracy = accuracy
        solution.execution_time = exec_time
        #print(completed)
        return solution


In [ ]:
import os
import sys
import random
import copy
import time

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier as KNN

#from utils import feature_selection, transfer_learning
#from utils.feature_selection import *
#from utils.transfer_learning import *
#from AAPSO import *

import warnings
warnings.filterwarnings('ignore')
data_directory = './'
epochs = 10
batch_size = 64
learning_rate = 0.0001
stepLR = 5
momentum = 0.9

print(f"data_directory: {data_directory}, epochs: {epochs}, batch_size: {batch_size}, learning_rate: {learning_rate}, stepLR: {stepLR}, momentum: {momentum}")
DIR_PATH = " /content/drive/MyDrive/chest_xray/chest_xray"# enter directory path for dataset
if DIR_PATH[-1]=='/':
    DIR_PATH = DIR_PATH[:-1]
TRAIN_DIR_PATH = os.path.join(DIR_PATH, 'train')
VAL_DIR_PATH = os.path.join(DIR_PATH, 'test')
print(TRAIN_DIR_PATH )
print(VAL_DIR_PATH )
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
print(mean)
print(std)
transformations = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine(
            degrees=(-180, 180), translate=(0.1, 0.1), scale=(0.9, 1.1),shear=(-5, 5)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}
train_dataset = torchvision.datasets.ImageFolder("/content/drive/MyDrive/chest_xray/train", transform=transformations['train'])
val_dataset = torchvision.datasets.ImageFolder("/content/drive/MyDrive/chest_xray/test", transform=transformations['val'])

classes_to_idx = train_dataset.class_to_idx
print(classes_to_idx)
train_batch_size = batch_size
learning_rate = learning_rate
num_classes = len(classes_to_idx)
num_epochs = epochs
print(train_batch_size)
print(learning_rate)
print(num_classes )
print(num_epochs)
phases = ['training', 'validation']

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# dataloaders
data_loader = {
    'training': DataLoader(dataset=train_dataset,
                           batch_size=train_batch_size,
                           shuffle=True,
                           num_workers=4),

    'validation': DataLoader(dataset=val_dataset,
                             batch_size=1,
                             shuffle=False,
                             num_workers=4)
}
for phase in phases:
    print(f'Length of {phase} loader = {len(data_loader[phase])}')


data_directory: ./, epochs: 10, batch_size: 64, learning_rate: 0.0001, stepLR: 5, momentum: 0.9
 /content/drive/MyDrive/chest_xray/chest_xray/train
 /content/drive/MyDrive/chest_xray/chest_xray/test
[0.485 0.456 0.406]
[0.229 0.224 0.225]
{'NORMAL': 0, 'PNEUMONIA': 1}
64
0.0001
2
10
Length of training loader = 82
Length of validation loader = 624


In [ ]:
model = torchvision.models.resnet50(pretrained=True)
model = model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model = ConvNet(model, num_classes)
model = model.to(device)
print(model)


ConvNet(
  (base_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(6

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=stepLR, gamma=0.1)
print(criterion)
print(optimizer)
print(exp_lr_scheduler)

CrossEntropyLoss()
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.0001
    lr: 0.0001
    maximize: False
    weight_decay: 0
)


In [ ]:
start = time.time()
model, history = train_model(model, criterion, optimizer, exp_lr_scheduler, data_loader, train_batch_size, num_epochs)
duration = time.time() - start
print(f'Training complete i+ {(duration // 60):.0f}mins {(duration % 60):.0f}s')


Epoch: [1/10] Phase: training | Loss: 11.715781 Accuracy: 0.913110
Epoch: [1/10] Phase: validation | Loss: 0.279617 Accuracy: 0.897436
====> Best accuracy reached so far at Epoch 1 Accuracy = 0.897436
-------------------------------------------------------------------------
Epoch: [2/10] Phase: training | Loss: 6.699223 Accuracy: 0.956936
Epoch: [2/10] Phase: validation | Loss: 0.330472 Accuracy: 0.886218
-------------------------------------------------------------------------
Epoch: [3/10] Phase: training | Loss: 4.660600 Accuracy: 0.967226
Epoch: [3/10] Phase: validation | Loss: 0.691758 Accuracy: 0.788462
-------------------------------------------------------------------------
Epoch: [4/10] Phase: training | Loss: 4.415251 Accuracy: 0.966273
Epoch: [4/10] Phase: validation | Loss: 0.308005 Accuracy: 0.889423
-------------------------------------------------------------------------
Epoch: [5/10] Phase: training | Loss: 3.604842 Accuracy: 0.973133
Epoch: [5/10] Phase: validation | L

In [ ]:
features = []
true_labels = []
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=1,
                          shuffle=False,
                          num_workers=4)
features1 = []
true_labels1 = []
test_loader = DataLoader(dataset=val_dataset,
                          batch_size=1,
                          shuffle=False,
                          num_workers=4)


In [ ]:
features, true_labels = eval_model_extract_features(features, true_labels, model, dataloader=train_loader, phase='training')
print(features)
print(true_labels)
features1, true_labels1 = eval_model_extract_features(features1, true_labels1, model, dataloader=test_loader, phase='validation')
print(features1)
print(true_labels1)

Accuracy of model on training set = 49.5590 %
tensor([[1285.,   56.],
        [2575., 1300.]])
tensor([0.9582, 0.3355])


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
X, y = get_features(features, true_labels)
print("for training set")
print(X)
print(y)
X1, y1 = get_features(features1, true_labels1)
print("for testing set")
print(X1)
print(y1)

for training set
[[-2.86007789e-03  1.22116394e-01 -6.53084787e-03 ... -4.00624936e-03
  -9.48431843e-04 -5.24168182e-03]
 [ 2.41724253e-01  6.81651682e-02 -4.18905681e-03 ... -6.88594813e-03
  -9.71884758e-04 -7.98641797e-03]
 [-1.15729065e-03  1.68605611e-01 -5.86543465e-03 ... -5.61060337e-03
   2.62557745e-01 -1.33937376e-03]
 ...
 [-1.16987445e-03  2.29671344e-01 -5.94809232e-03 ... -4.58241953e-03
   3.14673558e-02 -3.49536235e-03]
 [ 3.83936465e-01  1.86288550e-01 -3.68435145e-03 ...  2.71709524e-02
  -2.81993090e-03 -6.38285279e-03]
 [ 3.31333905e-01  2.13959888e-01 -2.35093990e-03 ... -3.59635160e-04
  -1.40392105e-04 -7.12235086e-03]]
[0 0 0 ... 1 1 1]
for testing set
[[ 2.0478873e-02  3.0814472e-01 -5.1401155e-03 ... -3.3940519e-03
   2.8230898e-02 -5.4338542e-03]
 [-2.1027298e-04  1.7039482e-01 -3.9838506e-03 ... -3.8165206e-03
  -1.3684290e-03 -7.2722253e-03]
 [-5.0929398e-04  4.4873509e-01 -5.7116249e-03 ... -2.5449248e-03
   9.3337521e-03 -5.5877366e-03]
 ...
 [ 9.164223

In [ ]:
fs_soln = AAPSO(num_agents=30, max_iter=20, train_data=X, train_label=y)
print(fs_soln)
agent = fs_soln.best_agent.copy()
print(agent)


                          Iteration - 1


Number of agents: 30

------------- Best Agent ---------------
Fitness: 0.903870629789272
Accuracy: 0.9090038314176245
Number of Features: 178
----------------------------------------

Particle 1 - Fitness: 0.903870629789272,Accuracy: 0.9090038314176245, Number of Features: 178
Particle 2 - Fitness: 0.9011765146072797,Accuracy: 0.9099616858237548, Number of Features: 271
Particle 3 - Fitness: 0.8974607878352491,Accuracy: 0.9061302681992337, Number of Features: 270
Particle 4 - Fitness: 0.8961669360632184,Accuracy: 0.9022988505747126, Number of Features: 207
Particle 5 - Fitness: 0.8957751137452107,Accuracy: 0.9013409961685823, Number of Features: 193
Particle 6 - Fitness: 0.8943688637452107,Accuracy: 0.9013409961685823, Number of Features: 229
Particle 7 - Fitness: 0.894254070881226,Accuracy: 0.9032567049808429, Number of Features: 280
Particle 8 - Fitness: 0.8930383441091955,Accuracy: 0.8994252873563219, Number of Features: 215
Particle 9 - F

In [ ]:
print("training set")
validate_FS(X, y, agent)
print("testing set")
validate_FS(X1, y1, agent)
